In [76]:
import os, mlflow
from dotenv import load_dotenv

load_dotenv(override=True)  # Carga las variables del archivo .env
EXPERIMENT_NAME = "/Users/priscila.cervantes@iteso.mx/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

2025/10/28 21:33:41 INFO mlflow.tracking.fluent: Experiment with name '/Users/priscila.cervantes@iteso.mx/nyc-taxi-experiments' does not exist. Creating a new experiment.


## Train Model

In [77]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [78]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [79]:
df_train = read_dataframe('C:/Users/prisc/apps/nyc-taxi-predictions-2025/data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('C:/Users/prisc/apps/nyc-taxi-predictions-2025/data/green_tripdata_2025-02.parquet')

In [80]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
categorical = ['PU_DO']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

In [81]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [82]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

## Optimización de hiperparámetros usando Optuna

In [83]:
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature

In [84]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

## Función objetivo

In [85]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna
#    - Recibe un `trial`, que se usa para proponer hiperparámetros.
#    - Entrena un modelo con esos hiperparámetros.
#    - Calcula la métrica de validación (RMSE) y la retorna (Optuna la minimizará).
#    - Abrimos un run anidado de MLflow para registrar cada trial.
# ------------------------------------------------------------
def objective(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha",   math.exp(-5), math.exp(-1), log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
        "objective": "reg:squarederror",  
        "seed": 42,                      
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "xgboost")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento con early stopping en el conjunto de validación
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=10,
        )

        # Predicción y métrica en validación
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val, y_pred)

        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.xgboost.log_model(
            booster,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )

    # Optuna minimiza el valor retornado
    return rmse

## Flujo de búsqueda

In [86]:
mlflow.xgboost.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=10)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "xgboost",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=10,
    )

    # Evaluar y registrar la métrica final en validación
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

    # Para que las longitudes coincidan, usa el mismo slice en y_pred
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.xgboost.log_model(
        booster,
        name="model",
        input_example=input_example,
        signature=signature
    )

[I 2025-10-28 21:34:23,417] A new study created in memory with name: no-name-54c6faa4-9dde-4b5f-82e5-960ea75f07cf


[0]	validation-rmse:5.72427
[1]	validation-rmse:5.57860
[2]	validation-rmse:5.56409
[3]	validation-rmse:5.56982
[4]	validation-rmse:5.57347
[5]	validation-rmse:5.55585
[6]	validation-rmse:5.55736
[7]	validation-rmse:5.55253
[8]	validation-rmse:5.55232
[9]	validation-rmse:5.53322
[10]	validation-rmse:5.53156
[11]	validation-rmse:5.53006
[12]	validation-rmse:5.52808
[13]	validation-rmse:5.52782
[14]	validation-rmse:5.52451
[15]	validation-rmse:5.52324
[16]	validation-rmse:5.52269
[17]	validation-rmse:5.52229
[18]	validation-rmse:5.52366
[19]	validation-rmse:5.52900
[20]	validation-rmse:5.52988
[21]	validation-rmse:5.52969
[22]	validation-rmse:5.52872
[23]	validation-rmse:5.53195
[24]	validation-rmse:5.52894
[25]	validation-rmse:5.52702
[26]	validation-rmse:5.53066


c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:36:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/28 21:37:18 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:37:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-28 21:37:23,247] Trial 0 finished with value: 5.5334711429183985 and parameters: {'max_depth': 40, 'learning_rate': 0.8625543817410922, 'reg_alpha': 0.12593061066249622, 'reg_lambda': 0.049454235173237264, 'min_child_weight': 0.6866535292359801}. Best is trial 0 with value: 5.5334711429183985.


🏃 View run secretive-skink-307 at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/3999049734856042/runs/cdecf9bb570b46d7ac946c6d11d2e2fa
🧪 View experiment at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/3999049734856042
[0]	validation-rmse:8.77707
[1]	validation-rmse:8.47452
[2]	validation-rmse:8.19669
[3]	validation-rmse:7.94182
[4]	validation-rmse:7.70901
[5]	validation-rmse:7.49664
[6]	validation-rmse:7.30245
[7]	validation-rmse:7.12626
[8]	validation-rmse:6.96581
[9]	validation-rmse:6.81994
[10]	validation-rmse:6.68775
[11]	validation-rmse:6.56832
[12]	validation-rmse:6.46039
[13]	validation-rmse:6.36272
[14]	validation-rmse:6.27449
[15]	validation-rmse:6.19394
[16]	validation-rmse:6.12203
[17]	validation-rmse:6.05669
[18]	validation-rmse:5.99838
[19]	validation-rmse:5.94487
[20]	validation-rmse:5.89709
[21]	validation-rmse:5.85472
[22]	validation-rmse:5.81565
[23]	validation-rmse:5.78117
[24]	validation-rmse:5.74956
[25]	validation-rmse:5.72125
[26

...

c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:49:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-28 21:49:15,162] Trial 9 finished with value: 5.460277869585555 and parameters: {'max_depth': 68, 'learning_rate': 0.1268351874747755, 'reg_alpha': 0.05394836382863035, 'reg_lambda': 0.03814164293595655, 'min_child_weight': 0.7706028272535065}. Best is trial 4 with value: 5.350864717798953.


🏃 View run likeable-boar-889 at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/3999049734856042/runs/5ed4949aba144c1cb8e4f76b06998993
🧪 View experiment at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/3999049734856042
[0]	validation-rmse:8.66900
[1]	validation-rmse:8.27724
[2]	validation-rmse:7.92728
[3]	validation-rmse:7.61583
[4]	validation-rmse:7.33855
[5]	validation-rmse:7.09228
[6]	validation-rmse:6.87486
[7]	validation-rmse:6.68312
[8]	validation-rmse:6.51512
[9]	validation-rmse:6.36795
[10]	validation-rmse:6.23874
[11]	validation-rmse:6.12574
[12]	validation-rmse:6.02624
[13]	validation-rmse:5.93991
[14]	validation-rmse:5.86418
[15]	validation-rmse:5.79827
[16]	validation-rmse:5.74190
[17]	validation-rmse:5.69254
[18]	validation-rmse:5.65012
[19]	validation-rmse:5.61385
[20]	validation-rmse:5.58229
[21]	validation-rmse:5.55495
[22]	validation-rmse:5.53061
[23]	validation-rmse:5.50968
[24]	validation-rmse:5.49174
[25]	validation-rmse:5.47625
[26]	

c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:50:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/28 21:50:48 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:50:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


🏃 View run XGBoost Hyperparameter Optimization (Optuna) at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/3999049734856042/runs/1c88711fc4ae4d0db27b73ef9f0f68ea
🧪 View experiment at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/3999049734856042


## Registrar modelo en 'Model Registry'

### Método 1: Manual

In [87]:
model_name = "workspace.default.nyc-taxi-model"

In [90]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="workspace.default.nyc-taxi-model"
)

Registered model 'workspace.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/28 21:51:58 WARNING mlflow.tracking._model_registry.fluent: Run with id b1e94feef25e4e2fa49cc01e302b3efb has no artifacts at artifact path 'model', registering model based on models:/m-5746a243e69c4ab996b82f3d19df9f3e instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '11' of model 'workspace.default.nyc-taxi-model'.


### Método 2: Automático

In [89]:
runs = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    order_by=["metrics.rmse ASC"],
    output_format="list"
)

# Obtener el mejor run
if len(runs) > 0:
    best_run = runs[0]
    print("🏆 Champion Run encontrado:")
    print(f"Run ID: {best_run.info.run_id}")
    print(f"RMSE: {best_run.data.metrics['rmse']}")
    print(f"Params: {best_run.data.params}")
else:
    print("⚠️ No se encontraron runs con métrica RMSE.")

🏆 Champion Run encontrado:
Run ID: b1e94feef25e4e2fa49cc01e302b3efb
RMSE: 5.350864717798953
Params: {'custom_metric': 'None', 'early_stopping_rounds': '10', 'learning_rate': '0.07565903471570516', 'max_depth': '63', 'maximize': 'None', 'min_child_weight': '2.2802382585441565', 'num_boost_round': '100', 'objective': 'reg:squarederror', 'reg_alpha': '0.021678779375600917', 'reg_lambda': '0.015480241912324163', 'seed': '42', 'verbose_eval': 'True'}


In [91]:
run_id = best_run.info.run_id

In [92]:
result = mlflow.register_model(
    model_uri=f"runs:/{best_run.info.run_id}/model",
    name=model_name
)

Registered model 'workspace.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/28 21:52:25 WARNING mlflow.tracking._model_registry.fluent: Run with id b1e94feef25e4e2fa49cc01e302b3efb has no artifacts at artifact path 'model', registering model based on models:/m-5746a243e69c4ab996b82f3d19df9f3e instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '12' of model 'workspace.default.nyc-taxi-model'.


## Asignar alias

In [93]:
from mlflow import MlflowClient

client = MlflowClient()

In [94]:
model_version = result.version
new_alias = "Champion"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

In [95]:
from datetime import datetime

date = datetime.today()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1761709949240, current_stage=None, deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description=('The model version 12 was transitioned to Champion on 2025-10-28 '
 '21:53:19.804600'), last_updated_timestamp=1761709996374, metrics=[<Metric: dataset_digest='', dataset_name='', key='best_iteration', model_id='m-5746a243e69c4ab996b82f3d19df9f3e', run_id='b1e94feef25e4e2fa49cc01e302b3efb', step=0, timestamp=1761709214676, value=96.0>,
 <Metric: dataset_digest='', dataset_name='', key='rmse', model_id='m-5746a243e69c4ab996b82f3d19df9f3e', run_id='b1e94feef25e4e2fa49cc01e302b3efb', step=0, timestamp=1761709232081, value=5.350864717798953>,
 <Metric: dataset_digest='', dataset_name='', key='stopped_iteration', model_id='m-5746a243e69c4ab996b82f3d19df9f3e', run_id='b1e94feef25e4e2fa49cc01e302b3

## Obteniendo modelos de model registry

In [96]:
import mlflow.pyfunc

model_version_uri = f"models:/{model_name}@Champion"

champion_version = mlflow.pyfunc.load_model(model_version_uri)
champion_version.predict(X_val)

c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:53:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


array([ 5.711086, 26.219116, 28.056229, ..., 21.681288, 15.960737,
       22.701014], shape=(44218,), dtype=float32)